In [1]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train')
import pandas as pd

In [15]:
df = pd.DataFrame({'text':newsgroups_train.data, 'target':newsgroups_train.target})

In [16]:
df.head()

,text,target
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14


In [ ]:
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import multiprocessing 
cores = multiprocessing.cpu_count()

### Word2vec Example 

![](params.png)

In [78]:
w2v_model = Word2Vec(min_count=20,
                     window=2,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

In [79]:
w2v_model.build_vocab([i.split() for i in df1.text], progress_per=10000)
w2v_model.train([i.split() for i in df1.text], total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)
w2v_model.init_sims(replace=True)

In [80]:
w2v_model.wv.most_similar(positive=["machine"])

[('build', 0.9571606516838074),
 ('fast', 0.9557944536209106),
 ("I've", 0.9378416538238525),
 ('need', 0.9350886344909668),
 ('bit', 0.933015763759613),
 ('windows', 0.9257797002792358),
 ('am', 0.9245187044143677),
 ('maybe', 0.922695517539978),
 ('uses', 0.9192616939544678),
 ('Apple', 0.9149423837661743)]

In [83]:
len(w2v_model.wv['machine'])

300

[intersting source](https://www.kaggle.com/pierremegret/gensim-word2vec-tutorial)

### Doc2vec Example

In [19]:
data = ["I love machine learning. Its awesome.",
        "I love coding in python",
        "I love building chatbots",
        "they chat amagingly well"]
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data)]

dm defines the training algorithm. If dm=1 means ‘distributed memory’ (PV-DM) and dm =0 means ‘distributed bag of words’ (PV-DBOW). Distributed Memory model preserves the word order in a document whereas Distributed Bag of words just uses the bag of words approach, which doesn’t preserve any word order.

In [20]:
max_epochs = 100
vec_size = 20
alpha = 0.025

model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm=1)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

/home/serge/Desktop/nlp/lib/python3.7/site-packages/gensim/models/doc2vec.py:574: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
/home/serge/Desktop/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration

In [28]:
model.infer_vector("I love chatbots".lower().split())

array([-8.6066397e-03, -1.8985610e-02, -1.6915781e-02, -1.2382539e-02,
        2.1471249e-02, -1.8044667e-02, -1.9045342e-02, -1.7452616e-02,
        5.6160032e-03,  5.2611856e-03,  1.1396936e-02,  1.7015431e-02,
        2.5557077e-02,  1.7080329e-02, -2.0881791e-02, -1.1690328e-02,
        2.3251973e-02, -9.5925508e-03, -8.1369508e-06,  1.7668009e-02],
      dtype=float32)

[worth visiting](https://medium.com/@mishra.thedeepak/doc2vec-simple-implementation-example-df2afbbfbad5)

### Problem

---
1) векторизовать тексты с помощью tf-idf <br>
2) векторизовать тексты с помощью word2vec(doc2vec) <br>
3) обучить классификатор (любой, например knn) <br>
4) сравнить качество, и ответить на вопрос, улучшает ли векоризация качество модели